In [1]:
import json
import torch
import random
import re
import string
import nltk
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from bs4 import BeautifulSoup
import torch
from transformers import BertModel, BertTokenizer, XLNetModel, XLNetTokenizer, T5EncoderModel, T5Tokenizer
from sklearn.model_selection import train_test_split
import re
import os
import requests
import gc
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from transformers import RobertaTokenizer, RobertaModel
from transformers import GPT2Tokenizer, GPT2Model
from transformers import T5Tokenizer, T5EncoderModel
from transformers import XLNetTokenizer, XLNetModel

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import f1_score, accuracy_score

from nltk.corpus import stopwords

nltk.download('stopwords')

import warnings
warnings.filterwarnings("ignore")


/home/numaan.naeem/.conda/envs/ibex/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/numaan.naeem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train_path = "/home/numaan.naeem/BEA_2025/mrbench_v3_devset.json"
test_path  = "/home/numaan.naeem/BEA_2025/mrbench_v3_testset.json"

In [3]:
# -------------------------- Load and preprocess data -------------------------- #
def load_and_process_train_data(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    records = []
    for item in data:
        conv_id = item["conversation_id"]
        history = item["conversation_history"]
        for model, details in item["tutor_responses"].items():
            record = {
                "conversation_id": conv_id,
                "model": model,
                "conversation_history": history,
                "response": details["response"],
                "mistake_identification": details["annotation"]["Mistake_Identification"].lower(),
            }
            records.append(record)

    return pd.DataFrame(records)

def load_and_process_test_data(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    records = []
    for item in data:
        conv_id = item["conversation_id"]
        history = item["conversation_history"]
        for model, details in item["tutor_responses"].items():
            record = {
                "conversation_id": conv_id,
                "model": model,
                "conversation_history": history,
                "response": details["response"]
            }
            records.append(record)

    return pd.DataFrame(records)

In [4]:
df_train = load_and_process_train_data(train_path)
df_test  = load_and_process_test_data(test_path)

In [5]:
df_train.head(8)

,conversation_id,model,conversation_history,response,mistake_identification
0,221-362eb11a-f190-42a6-b2a4-985fafdcfa9e,Sonnet,"Tutor: Hi, could you please provide a step-by-...","Great, you've correctly identified the cost of...",yes
1,221-362eb11a-f190-42a6-b2a4-985fafdcfa9e,Llama318B,"Tutor: Hi, could you please provide a step-by-...",Now that we know the cost of 1 pound of meat i...,yes
2,221-362eb11a-f190-42a6-b2a4-985fafdcfa9e,Llama31405B,"Tutor: Hi, could you please provide a step-by-...","You're close, but I notice that you calculated...",yes
3,221-362eb11a-f190-42a6-b2a4-985fafdcfa9e,GPT4,"Tutor: Hi, could you please provide a step-by-...","That's correct. So, if 1 pound of meat costs $...",yes
4,221-362eb11a-f190-42a6-b2a4-985fafdcfa9e,Mistral,"Tutor: Hi, could you please provide a step-by-...",It seems like you've calculated the cost as if...,yes
5,221-362eb11a-f190-42a6-b2a4-985fafdcfa9e,Expert,"Tutor: Hi, could you please provide a step-by-...","And if there are 5 sandwiches, what is the mea...",yes
6,221-362eb11a-f190-42a6-b2a4-985fafdcfa9e,Gemini,"Tutor: Hi, could you please provide a step-by-...",You're absolutely right about the cost of meat...,yes
7,221-362eb11a-f190-42a6-b2a4-985fafdcfa9e,Phi3,"Tutor: Hi, could you please provide a step-by-...","To find the area of a rectangle, multiply its ...",no


In [6]:
df_test.head(8)

,conversation_id,model,conversation_history,response
0,1030-adb61831-0383-4e51-a673-ab978590f69b,Tutor_1,"Tutor: Hi, could you please provide a step-by-...",It looks like you've done a great job figuring...
1,1030-adb61831-0383-4e51-a673-ab978590f69b,Tutor_2,"Tutor: Hi, could you please provide a step-by-...","You've done a great job, but there's a small m..."
2,1030-adb61831-0383-4e51-a673-ab978590f69b,Tutor_3,"Tutor: Hi, could you please provide a step-by-...","OK, read the question again, and answer these ..."
3,1030-adb61831-0383-4e51-a673-ab978590f69b,Tutor_4,"Tutor: Hi, could you please provide a step-by-...","Tutor: I see where you're coming from, but I t..."
4,1030-adb61831-0383-4e51-a673-ab978590f69b,Tutor_5,"Tutor: Hi, could you please provide a step-by-...",Great job! Can you explain how you arrived at ...
5,1030-adb61831-0383-4e51-a673-ab978590f69b,Tutor_6,"Tutor: Hi, could you please provide a step-by-...","I appreciate your clear explanation, but let's..."
6,1030-adb61831-0383-4e51-a673-ab978590f69b,Tutor_7,"Tutor: Hi, could you please provide a step-by-...","It sounds like you're on the right track, but ..."
7,1030-adb61831-0383-4e51-a673-ab978590f69b,Tutor_8,"Tutor: Hi, could you please provide a step-by-...",Your method is absolutely correct - multiplyin...


In [7]:
df_train.shape, df_test.shape

((2476, 5), (1547, 4))

In [8]:
df_train["mistake_identification"].value_counts()

mistake_identification
yes               1932
no                 370
to some extent     174
Name: count, dtype: int64

In [9]:
def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text


def clean_contractions(text):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_text(text)
    text = clean_contractions(text)
    text = remove_space(text)
    # text = remove_stopwords(text)  # Added stopword removal step
    return text

def remove_stopwords(text):
    '''Removes stopwords from the text.'''
    stop_words = set(stopwords.words('english'))
    text = text.split()
    text = [word for word in text if word not in stop_words]
    return " ".join(text)

# Training data preprocessing
df_train['conversation_history_processed'] = df_train['conversation_history'].apply(text_preprocessing_pipeline)
df_train['response_processed'] = df_train['response'].apply(text_preprocessing_pipeline)

df_train['conversation_history_processed'] = df_train['conversation_history_processed'].apply(remove_stopwords)
df_train['response_processed'] = df_train['response_processed'].apply(remove_stopwords)

# Test data preprocessing
df_test['conversation_history_processed'] = df_test['conversation_history'].apply(text_preprocessing_pipeline)
df_test['response_processed'] = df_test['response'].apply(text_preprocessing_pipeline)

df_test['conversation_history_processed'] = df_test['conversation_history_processed'].apply(remove_stopwords)
df_test['response_processed'] = df_test['response_processed'].apply(remove_stopwords)

In [10]:
label_map = {'no': 0, 'yes': 1, 'to some extent': 2}

In [11]:
df_train['mistake_identification'] = df_train['mistake_identification'].str.strip().str.lower().map(label_map)

In [12]:
X_train = df_train.drop(columns=["mistake_identification"])
y_train = df_train["mistake_identification"]

In [13]:
X_train.shape, y_train.shape

((2476, 6), (2476,))

In [14]:
df_train.shape

(2476, 7)

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train.shape, X_val.shape, df_test.shape, y_train.shape, y_val.shape

((1980, 6), (496, 6), (1547, 6), (1980,), (496,))

In [16]:
class LM_EMBED:

    def __init__(self, language_model, max_len):
        self.lang_model = language_model
        self.max_len = max_len

        # Import tokenizer and model based on the specified language model
        if self.lang_model == 'BERT':
            self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
            self.model = BertModel.from_pretrained('bert-base-uncased')
        elif self.lang_model == 'ROBERTA':
            self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
            self.model = RobertaModel.from_pretrained('roberta-base')
        elif self.lang_model == 'GPT2':
            self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
            self.model = GPT2Model.from_pretrained('gpt2')
            # Assign a padding token if not already present
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.model.config.pad_token_id = self.tokenizer.pad_token_id
        elif self.lang_model == 'T5':
            self.tokenizer = T5Tokenizer.from_pretrained('t5-base')
            self.model = T5EncoderModel.from_pretrained('t5-base')
        elif self.lang_model == 'XLNET':
            self.tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
            self.model = XLNetModel.from_pretrained('xlnet-base-cased')
        else:
            raise ValueError(f"Unsupported language model: {self.lang_model}")

    def extract_word_embs(self, text_df, filename, col_name):
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {device}")

        self.model = self.model.to(device)
        self.model.eval()

        texts = text_df[col_name].tolist()

        # Tokenize and encode the texts
        tokens = self.tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            padding='max_length',
            max_length=self.max_len,
            truncation=True,
            return_tensors='pt'
        )
        input_ids = tokens['input_ids'].to(device)
        attention_mask = tokens['attention_mask'].to(device)

        torch.cuda.empty_cache()

        for i in range(0, len(input_ids), 10):
            if i % 100 == 0:
                print(f"Processing batch ending at {i}")
            with torch.no_grad():
                embeddings = self.model(input_ids=input_ids[i:i+10],
                                        attention_mask=attention_mask[i:i+10])[0]
                emb_array = embeddings.cpu().numpy()

                if i == 0:
                    embedding_res = emb_array
                else:
                    embedding_res = np.concatenate((embedding_res, emb_array))

        features = self.extract_features(embedding_res, attention_mask)
        padded_arr = self.pad(features)

        print("Saving embeddings...")
        np.save(filename, padded_arr)

    def extract_features(self, emb_res, att_msk):
        features = []
        for seq_num in range(len(emb_res)):
            seq_len = (att_msk[seq_num] == 1).sum()

            if self.lang_model in ['BERT', 'ROBERTA']:
                seq_emb = emb_res[seq_num][1:seq_len-1]
            elif self.lang_model == 'T5':
                seq_emb = emb_res[seq_num][:seq_len-1]
            elif self.lang_model == 'XLNET':
                padded_seq_len = len(att_msk[seq_num])
                seq_emb = emb_res[seq_num][padded_seq_len-seq_len:padded_seq_len-2]
            elif self.lang_model == 'GPT2':
                seq_emb = emb_res[seq_num][:seq_len]
            else:
                raise ValueError(f"Unsupported language model: {self.lang_model}")

            features.append(seq_emb)
        return np.array(features, dtype=object)

    def pad(self, features):
        dim1 = self.max_len
        dim2 = features[0].shape[1]

        for i, feature in enumerate(features):
            if i % 100 == 0:
                print(f"Padding batch {i}")

            padded_feature = np.zeros((dim1, dim2))
            padded_feature[:feature.shape[0], :feature.shape[1]] = feature

            if i == 0:
                padded_arr = padded_feature
            elif i == 1:
                padded_arr = np.stack((padded_arr, padded_feature), axis=0)
            else:
                reshaped_feature = padded_feature.reshape(1, dim1, dim2)
                padded_arr = np.vstack((padded_arr, reshaped_feature))

        return padded_arr

In [17]:
max_seq_len = 399

BERT = LM_EMBED('BERT', max_seq_len)

In [18]:
BERT.extract_word_embs(X_train, "BERT_EMBED_X_TRAIN_conv.npy", "conversation_history_processed")
BERT.extract_word_embs(X_val, "BERT_EMBED_X_VAL_conv.npy", "conversation_history_processed")
BERT.extract_word_embs(df_test, "BERT_EMBED_X_TEST_conv.npy", "conversation_history_processed")

BERT.extract_word_embs(X_train, "BERT_EMBED_X_TRAIN_resp.npy", "response_processed")
BERT.extract_word_embs(X_val, "BERT_EMBED_X_VAL_resp.npy", "response_processed")
BERT.extract_word_embs(df_test, "BERT_EMBED_X_TEST_resp.npy", "response_processed")

Using device: cuda:0
Processing batch ending at 0
Processing batch ending at 100
Processing batch ending at 200
Processing batch ending at 300
Processing batch ending at 400
Processing batch ending at 500
Processing batch ending at 600
Processing batch ending at 700
Processing batch ending at 800
Processing batch ending at 900
Processing batch ending at 1000
Processing batch ending at 1100
Processing batch ending at 1200
Processing batch ending at 1300
Processing batch ending at 1400
Processing batch ending at 1500
Processing batch ending at 1600
Processing batch ending at 1700
Processing batch ending at 1800
Processing batch ending at 1900
Padding batch 0
Padding batch 100
Padding batch 200
Padding batch 300
Padding batch 400
Padding batch 500
Padding batch 600
Padding batch 700
Padding batch 800
Padding batch 900
Padding batch 1000
Padding batch 1100
Padding batch 1200
Padding batch 1300
Padding batch 1400
Padding batch 1500
Padding batch 1600
Padding batch 1700
Padding batch 1800
Pa

In [1]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten, Dropout, InputLayer, Conv2D, MaxPooling2D, BatchNormalization, Input
import pandas as pd
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.ticker as mtick
from matplotlib import pyplot as plt

2025-04-20 22:00:26.543295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745172026.992102 1086914 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745172027.006864 1086914 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745172027.168858 1086914 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745172027.168874 1086914 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745172027.168876 1086914 computation_placer.cc:177] computation placer alr

In [2]:
def load_X_data(lang_model):
  if lang_model =='BERT':
    X_train_conv = np.load('BERT_EMBED_X_TRAIN_conv.npy')
    X_val_conv = np.load('BERT_EMBED_X_VAL_conv.npy')
    X_train_resp = np.load('BERT_EMBED_X_TRAIN_resp.npy')
    X_val_resp = np.load('BERT_EMBED_X_VAL_resp.npy')
  elif lang_model =='ROBERTA':
    X_train = np.load('ROBERTA_EMBED_X_TRAIN.npy')
    X_val = np.load('ROBERTA_EMBED_X_VAL.npy')
  elif lang_model =='T5':
    X_train = np.load('T5_EMBED_X_TRAIN.npy')
    X_val = np.load('T5_EMBED_X_VAL.npy')
  elif lang_model =='GPT2':
    X_train = np.load('T5XL_BFD_EMBED_X_TRAIN.npy')
    X_val = np.load('T5XL_BFD_EMBED_X_VAL.npy')
  elif lang_model =='XLNET':
    X_train = np.load('XLNET_EMMBED_X_TRAIN.npy')
    X_val = np.load('XLNET_EMMBED_X_VAL.npy')

  return X_train_conv, X_train_resp, X_val_conv, X_val_resp

In [3]:
def load_X_Test_data(lang_model):
  if lang_model =='BERT':
    print("bert")
    X_test_conv = np.load('BERT_EMBED_X_TEST_conv.npy')
    X_test_resp = np.load('BERT_EMBED_X_TEST_resp.npy')
  elif lang_model =='ROBERTA':
    print("here")
    X_test = np.load('ROBERTA_EMBED_X_TEST.npy')
  elif lang_model =='T5':
    print("here")
    X_test = np.load('T5_EMBED_X_TEST.npy')
  elif lang_model =='GPT2':
    X_test = np.load('T5_VELTRI_X_TEST_INDEP.npy')
  elif lang_model =='T5XL_BFD':
    print("T5XL_BFD")
    X_test = np.load('T5XL_BFD_EMBED_X_TEST.npy')
  elif lang_model =='XLNET':
    print("here")
    X_test = np.load('XLNET_EMMBED_X_TEST.npy')
  elif lang_model =='VELTRI':
    X_test = np.load('Datasets/VELTRI/X_test_VELTRI_Conv.npy')
  
  
  return X_test_conv, X_test_resp

In [4]:
X_train_conv, X_train_resp, X_val_conv, X_val_resp = load_X_data('BERT')
X_test_conv, X_test_resp = load_X_Test_data('BERT')

bert


In [25]:
def average_pool_embeddings(embeddings):
    """
    Applies average pooling to reduce the shape of embeddings from (N, L, D) to (N, D).
    
    Parameters:
        embeddings (numpy.ndarray): The input tensor of shape (N, L, D), where
                                     N is the number of samples,
                                     L is the max sequence length,
                                     D is the embedding dimension.
    
    Returns:
        numpy.ndarray: The output tensor of shape (N, D).
    """
    if len(embeddings.shape) != 3:
        raise ValueError("Input embeddings must have a shape of (N, L, D).")
    
    # Average pooling along the sequence length axis
    pooled_embeddings = np.mean(embeddings, axis=1)
    return pooled_embeddings

X_train_conv = average_pool_embeddings(X_train_conv)
X_val_conv = average_pool_embeddings(X_val_conv)
X_test_conv = average_pool_embeddings(X_test_conv)

X_train_resp = average_pool_embeddings(X_train_resp)
X_val_resp = average_pool_embeddings(X_val_resp)
X_test_resp = average_pool_embeddings(X_test_resp)

In [27]:
# Final averaged features
X_train = (X_train_conv + X_train_resp) / 2
X_val = (X_val_conv + X_val_resp) / 2
X_test = (X_test_conv + X_test_resp) / 2

In [28]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)

X_train shape: (1980, 768)
X_val shape: (496, 768)
X_test shape: (1547, 768)


In [58]:
import pandas as pd
import numpy as np
import re
import math

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesRegressor
from sklearn.model_selection import LeaveOneOut, cross_validate, cross_val_score, StratifiedShuffleSplit, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from sklearn.metrics import confusion_matrix
import math
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier  # Import Decision Tree
from sklearn.model_selection import GridSearchCV, LeaveOneOut  # Model selection tools

import joblib
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [59]:
def SVM(X,Y,x,y):
    param_grid = {
        'C': [0.01, 0.1, 1, 10, 100],  # Regularization parameter
        'kernel': ['linear', 'rbf', 'sigmoid'],  # Removed 'precomputed' kernel
    }
    svm=SVC(random_state=42, probability=True)
    svm_cv = GridSearchCV(svm, param_grid, scoring='accuracy', cv=KFold(10), n_jobs=4)
    
    # fitting the model for grid search
    svm_cv.fit(X,Y)
    svm=svm_cv.best_estimator_
    
    print('\n Test Results SVM\n')
    y_p=svm.predict(x)
    
    print("SVM:\n", classification_report(y, y_p))
    joblib.dump(svm, "svm_model.joblib")
    
    return svm

In [60]:
def Tree(X,Y,x,y):
    #create new a tree model
    DT=DecisionTreeClassifier(random_state=42)
    param_grid = {'max_depth':range(2,20),
                'criterion':['gini', 'entropy']}
    tree_cv = GridSearchCV(DT, param_grid, scoring='accuracy', cv=KFold(10), n_jobs=4)
    tree_cv.fit(X, Y)
    tree=tree_cv.best_estimator_
    
    print('\n Test Results Decision Tree\n')
    y_p=tree.predict(x)

    print("Tree:\n", classification_report(y, y_p))
    joblib.dump(tree, "tree_model.joblib")
    
    return tree

In [61]:
def NaiveBayes(X, Y, x, y):
    # Create a Naive Bayes model
    NB = GaussianNB()
    
    # Define hyperparameter grid (only 'var_smoothing' for GaussianNB)
    param_grid = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]}

    # GridSearchCV with Leave-One-Out Cross-Validation
    nb_cv = GridSearchCV(NB, param_grid, scoring='accuracy', cv=KFold(10), n_jobs=4)
    nb_cv.fit(X, Y)
    nb=nb_cv.best_estimator_
    
    print('\n Test Results MLP\n')
    y_p=nb.predict(x)

    print("NB:\n", classification_report(y, y_p))
    joblib.dump(nb, "nb_model.joblib")
    
    return nb

In [62]:
def KNN(X,Y,x,y):
    #create new a knn model
    knn = KNeighborsClassifier()
    #create a dictionary of all values we want to test for n_neighbors
    param_grid = {'n_neighbors': np.arange(1, 10),
                'metric': ['minkowski', 'euclidean', 'manhattan']}
    #use gridsearch to test all values for n_neighbors
    knn_cv = GridSearchCV(knn, param_grid, scoring='accuracy', cv=KFold(10), n_jobs=4)
    #fit model to data
    knn_cv.fit(X, Y)
    knn = knn_cv.best_estimator_
    
    print('\n Test Results KNN\n')
    y_p=knn.predict(x)
    
    print("NB:\n", classification_report(y, y_p))
    joblib.dump(knn, "knn_model.joblib")
    
    return knn

In [63]:
def LRegression(X,Y,x,y):
    #create new a tree model
    LR=LogisticRegression(random_state=42, max_iter=1500)
    param_grid = {'penalty':['l2', None]}
    
    lr_cv = GridSearchCV(LR, param_grid, scoring='accuracy', cv=KFold(10), n_jobs=4)
    lr_cv.fit(X, Y)
    lr=lr_cv.best_estimator_
    
    print('\n Test Results LR\n')
    y_p=lr.predict(x)
    
    print("NB:\n", classification_report(y, y_p))
    joblib.dump(lr, "lr_model.joblib")
    
    return lr

In [64]:
def RandomForest(X,Y,x,y):
    # Number of trees in random forest
    max_depth=range(2,20)
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt', 'log2']
    # Maximum number of levels in tree
    criterion = ['gini', 'entropy']
    
    random_grid = {'max_depth': max_depth,
                'criterion': criterion,
                'max_features': max_features}
    
    rf = RandomForestClassifier(random_state=42)
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                                cv=KFold(10), random_state=42, n_jobs = 4)
    rf_random.fit(X, Y)
    rf = rf_random.best_estimator_
    
    print('\n Test Results RF\n')
    y_p=rf.predict(x)

    print("RF:\n", classification_report(y, y_p))
    joblib.dump(rf, "rf_model.joblib")
    
    return rf

In [65]:
def ADABoost(X,Y,x,y):
    #create new a tree model
    Ada = AdaBoostClassifier(DecisionTreeClassifier(random_state=42), random_state=42)
    param_grid = {'estimator__max_depth' : [1, 2, 3, 4, 5],
                'estimator__criterion' : ['gini', 'entropy'],
                'estimator__splitter' :   ['best', 'random']}
    ada_cv = GridSearchCV(Ada, param_grid, cv=KFold(10), scoring = 'accuracy', n_jobs=4)
    ada_cv.fit(X,Y)
    ada=ada_cv.best_estimator_
    
    print('\n Test Results ADABoost\n')
    y_p=ada.predict(x)

    print("ADABoost:\n", classification_report(y, y_p))
    joblib.dump(rf, "ada_model.joblib")
    
    return ada

In [66]:
def MLPClassify(X,Y,x,y):
    #create new a tree model
    MLP=MLPClassifier(random_state=42, max_iter=1000)
    param_grid = {
      'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],  # Different sizes of hidden layers
      'activation': ['relu', 'tanh', 'logistic'],  # Activation functions
      'solver': ['adam', 'sgd'],  # Optimization algorithms
      'alpha': [0.0001, 0.001, 0.01],  # L2 regularization parameter
      'learning_rate': ['constant', 'adaptive']  # Learning rate schedule
    }
    
    mlp_cv = GridSearchCV(MLP, param_grid, scoring='accuracy', cv=KFold(10), n_jobs=4)
    mlp_cv.fit(X, Y)
    mlp=mlp_cv.best_estimator_
    
    print('\n Test Results MLP\n')
    y_p=mlp.predict(x)
    
    print("MLP:\n", classification_report(y, y_p))
    joblib.dump(rf, "mlp_model.joblib")
    
    return mlp

In [68]:
svm = SVM(X_train, y_train, X_val, y_val)


 Test Results SVM

SVM:
               precision    recall  f1-score   support

           0       0.74      0.43      0.55        67
           1       0.85      0.98      0.91       396
           2       1.00      0.03      0.06        33

    accuracy                           0.84       496
   macro avg       0.87      0.48      0.51       496
weighted avg       0.85      0.84      0.81       496



In [69]:
tree = Tree(X_train, y_train, X_val, y_val)


 Test Results Decision Tree

Tree:
               precision    recall  f1-score   support

           0       0.40      0.03      0.06        67
           1       0.80      0.99      0.89       396
           2       0.00      0.00      0.00        33

    accuracy                           0.80       496
   macro avg       0.40      0.34      0.31       496
weighted avg       0.69      0.80      0.72       496



In [70]:
nb = NaiveBayes(X_train, y_train, X_val, y_val)


 Test Results MLP

NB:
               precision    recall  f1-score   support

           0       0.16      0.19      0.18        67
           1       0.78      0.55      0.64       396
           2       0.09      0.36      0.14        33

    accuracy                           0.49       496
   macro avg       0.34      0.37      0.32       496
weighted avg       0.65      0.49      0.54       496



In [71]:
knn = KNN(X_train, y_train, X_val, y_val)


 Test Results KNN

NB:
               precision    recall  f1-score   support

           0       0.33      0.06      0.10        67
           1       0.82      0.99      0.90       396
           2       0.50      0.03      0.06        33

    accuracy                           0.80       496
   macro avg       0.55      0.36      0.35       496
weighted avg       0.73      0.80      0.73       496



In [72]:
lr = LRegression(X_train, y_train, X_val, y_val)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



 Test Results LR

NB:
               precision    recall  f1-score   support

           0       0.50      0.01      0.03        67
           1       0.80      1.00      0.89       396
           2       0.00      0.00      0.00        33

    accuracy                           0.80       496
   macro avg       0.43      0.34      0.31       496
weighted avg       0.71      0.80      0.71       496



In [73]:
rf = RandomForest(X_train, y_train, X_val, y_val)

/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor


 Test Results RF

RF:
               precision    recall  f1-score   support

           0       0.67      0.03      0.06        67
           1       0.81      1.00      0.89       396
           2       1.00      0.06      0.11        33

    accuracy                           0.81       496
   macro avg       0.82      0.36      0.35       496
weighted avg       0.80      0.81      0.73       496



In [74]:
ada = ADABoost(X_train, y_train, X_val, y_val)


 Test Results ADABoost

ADABoost:
               precision    recall  f1-score   support

           0       0.48      0.16      0.24        67
           1       0.82      0.97      0.89       396
           2       0.80      0.12      0.21        33

    accuracy                           0.80       496
   macro avg       0.70      0.42      0.45       496
weighted avg       0.77      0.80      0.76       496



In [75]:
mlp = MLPClassify(X_train, y_train, X_val, y_val)

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perce


 Test Results MLP

MLP:
               precision    recall  f1-score   support

           0       0.60      0.39      0.47        67
           1       0.85      0.96      0.90       396
           2       0.40      0.06      0.11        33

    accuracy                           0.82       496
   macro avg       0.62      0.47      0.49       496
weighted avg       0.79      0.82      0.79       496



In [ ]:
import pandas as pd

meta_train_df = pd.DataFrame({
    'svm': svm.predict_proba(X_train)[:, 1],
    'knn': knn.predict_proba(X_train)[:, 1],
    'nb': nb.predict_proba(X_train)[:, 1],
    'tree': tree.predict_proba(X_train)[:, 1],
    'rf': rf.predict_proba(X_train)[:, 1],
    'ada': ada.predict_proba(X_train)[:, 1],
    'mlp': mlp.predict_proba(X_train)[:, 1],
    'lr': lr.predict_proba(X_train)[:, 1],
})

meta_val_df = pd.DataFrame({
    'svm': svm.predict_proba(X_val)[:, 1],
    'knn': knn.predict_proba(X_val)[:, 1],
    'nb': nb.predict_proba(X_val)[:, 1],
    'tree': tree.predict_proba(X_val)[:, 1],
    'rf': rf.predict_proba(X_val)[:, 1],
    'ada': ada.predict_proba(X_val)[:, 1],
    'mlp': mlp.predict_proba(X_val)[:, 1],
    'lr': lr.predict_proba(X_val)[:, 1],
})

meta_test_df = pd.DataFrame({
    'svm': svm.predict_proba(X_test)[:, 1],
    'knn': knn.predict_proba(X_test)[:, 1],
    'nb': nb.predict_proba(X_test)[:, 1],
    'tree': tree.predict_proba(X_test)[:, 1],
    'rf': rf.predict_proba(X_test)[:, 1],
    'ada': ada.predict_proba(X_test)[:, 1],
    'map': map.predict_proba(X_test)[:, 1],
    'lr': lr.predict_proba(X_test)[:, 1],
})

In [ ]:
def LRegression(X,Y,x):
    #create new a tree model
    LR=LogisticRegression(random_state=42, max_iter=1500)
    param_grid = {'penalty':['l2', None]}
    
    lr_cv = GridSearchCV(LR, param_grid, scoring='accuracy', cv=KFold(10), n_jobs=4)
    lr_cv.fit(X, Y)
    lr=lr_cv.best_estimator_
    
    print('\n Test Results LR\n')
    y_p=lr.predict(x)

    return lr, yp

In [ ]:
lr, y_p_val = LRegression(meta_train_df, y_train, meta_val_df)
print("Meta Val:\n", classification_report(y_val, y_p_val))

In [ ]:
meta_full_df = pd.concat([meta_train_df, meta_val_df], axis=0).reset_index(drop=True)
y_full = np.concatenate([y_train, y_val])

In [ ]:
lr, y_p_test = LRegression(meta_full_df, y_full, meta_test_df)